In [ ]:
from pyXenium.io.xenium_gene_protein_loader import load_xenium_gene_protein
from pyXenium.analysis import ProteinMicroEnv
from pyXenium.vis.fig_microenv import build_microenv_figure

# 1) 载入数据
adata = load_xenium_gene_protein(
    base_path="~/proj/data/Xenium_V1_Human_Kidney_FFPE_Protein",
    clusters_relpath="analysis/clustering/gene_expression_graphclust/clusters.csv",
    cluster_column_name="cluster"
)

# 2) 运行微环境分析（如果你已运行过，可直接跳到第3步）
anl = ProteinMicroEnv(
    adata=adata,
    protein_obsm="protein",
    protein_norm_obsm="protein_norm",
    cluster_key="cluster",
    spatial_obsm="spatial",          # 若没有 'spatial'，会回退 x_centroid/y_centroid
    obs_xy=("x_centroid", "y_centroid"),
    random_state=0
)

cluster_id = "3"             # 替换为你的目标簇
protein    = "Ki-67"         # 替换为你的目标蛋白；比如 EPCAM/KRT8/KRT18/CD68/PDCD1 等
res = anl.analyze(
    cluster_id=cluster_id,
    protein=protein,
    group_key="cluster",      # 若已有 obs['cell_type']，优先用 'cell_type'
    radius=None,              # None 自动；也可给 40~60.0
    permutations=999,
    save_dir="./microenv_out"
)

# 3) 生成论文图板（PDF/PNG/SVG）
outbase = build_microenv_figure(
    adata=adata,
    res=res,
    cluster_id=cluster_id,
    protein=protein,
    group_key="cluster",      # 仅用于标题说明
    spatial_obsm="spatial",
    obs_xy=("x_centroid", "y_centroid"),
    outdir="./figures",
    basename=None,            # 用默认命名 Fig_microenv_cluster{cluster_id}_{protein}
    figsize_inches=(7.0, 5.0),
    scatter_s=0.4,            # 点的像素大小（大样本建议 0.3~0.5）
    scale_bar_um=100.0        # 如果坐标单位不是 μm，可改为 None 或正确单位长度
)

print("Saved:", outbase + ".pdf")
